In [15]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
import numpy as np
replacement_value = 'dummy_repalcement_value'
path = '/home/td/Documents'

# Problem structure

One way to set up the problem is to make it a binary classification task. This has the advantages of simplicity but it makes the assumption that large Makes share characteristics which is not always true. In this pass I solved the problem this way. 

A more thorough way to set up the problem would be to predict the make or a group of makes sharing similar characteristics and back into the probability. 

# Data problems

In [16]:
# Label problems
path = '/home/td/Documents'
df = pd.read_csv('{path}/tickets.csv'.format(path=path), low_memory=False)

top_25_makes = df['Make'].value_counts()[:25].index.tolist()
df['Make'].value_counts()[:25]

TOYT    721411
HOND    491961
FORD    382695
NISS    311324
CHEV    297076
BMW     199221
MERZ    177307
VOLK    149501
HYUN    133864
DODG    127764
LEXS    124508
KIA     101746
JEEP    100909
AUDI     84229
MAZD     79853
OTHR     72411
GMC      62391
CHRY     57317
INFI     56809
ACUR     52703
SUBA     46898
VOLV     42330
TOYO     40064
MITS     37842
CADI     34080
Name: Make, dtype: int64

Curation will be required, there is at least 1 duplicate with toyota and there are some invalid makes such as OTHR

In [17]:
df.loc[:,'Latitude'] = df.loc[:,'Latitude'].replace(99999.0, np.nan)
df.loc[:,'Longitude'] = df.loc[:,'Longitude'].replace(99999.0, np.nan)


In [18]:


percent_missing = df.isnull().sum()*100 / len(df)
missing_value_df = pd.DataFrame({'column_name': df.columns,
                                 'percent_missing': percent_missing})
missing_value_df

,column_name,percent_missing
Ticket number,Ticket number,0.000000
Issue Date,Issue Date,0.006143
Issue time,Issue time,0.029601
Meter Id,Meter Id,73.991538
Marked Time,Marked Time,96.669739
RP State Plate,RP State Plate,0.008767
Plate Expiry Date,Plate Expiry Date,9.108706
VIN,VIN,99.813099
Make,Make,50.062606
Body Style,Body Style,0.101879


There are problems with the latitudes and longitudes, they don't make to standard coordinates. For example: Record with ticket id 1109139006 should have latitude and longitude of 34.156940, -118.435250. It has 6467477, 1880027 instead. Provided lat and long don't seem to map to standard formats.


In [19]:
#TODO: check for differences in test and train set

# Label analysis

In [20]:
df_features = pd.read_csv('feature_analysis.csv')

In [101]:
df_features.sort_values('model_feature_importance', ascending = False)[:20]


,columns,intercept,model_feature_importance,p_value,r_value,slope,std_err
121,Fine amount,0.498704,0.022585,0.509448,0.001181,1.849256e-05,2.803185e-05
74,ticket_year_2017,0.499632,0.022433,0.486821,0.001245,1.422203e-03,2.045231e-03
72,ticket_year_2015,0.500049,0.020023,0.924127,-0.000171,-1.971760e-04,2.070377e-03
91,ticket_dow_2,0.500216,0.018904,0.611999,-0.000908,-1.169437e-03,2.305569e-03
123,plate_expiration_diff_ts,0.499560,0.018706,0.257164,-0.002029,-3.013163e-22,2.659169e-22
75,ticket_year_2018,0.499675,0.018495,0.506290,0.001190,1.415802e-03,2.130223e-03
2,color_BK,0.499930,0.018190,0.882280,0.000265,3.224668e-04,2.177657e-03
73,ticket_year_2016,0.500465,0.017299,0.370883,-0.001603,-1.844952e-03,2.061809e-03
16,agency_53.0,0.500113,0.016532,0.787009,-0.000484,-6.293665e-04,2.329293e-03
93,ticket_dow_4,0.500748,0.016097,0.057241,-0.003405,-4.623488e-03,2.431534e-03


In [22]:
df_labels = pd.read_csv('label_analysis.csv')
df_labels = df_labels[df_labels['Make'].isin(top_25_makes)]
df_counts= df_labels['Make'].value_counts()
from sklearn.preprocessing import StandardScaler

relevant_features = ['Fine amount', 
                     'plate_expiration_diff_ts', 
                     'lat_long_outlier_score', 
                     'route_dummy_replacement_value',
                     'color_GY',
                     'color_BL',
                     'color_BK',
                     'color_WT',
                     'color_BL',
                     'color_SL',
                     'route_00600',
                     'route_00500']


for i in relevant_features:
    scaler = StandardScaler()
    df_labels.loc[:, i] =  scaler.fit_transform(df_labels[i].values.reshape(-1, 1))
df_labels


df_group = df_labels[['Make'] + relevant_features].groupby('Make').mean()

/home/td/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/home/td/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [23]:
from sklearn.cluster import KMeans


cluster_alg = KMeans(n_clusters = 2)
df_group['cluster'] = cluster_alg.fit_predict(df_group[relevant_features])
df_group.sort_values('cluster')


,Fine amount,plate_expiration_diff_ts,lat_long_outlier_score,route_dummy_replacement_value,color_GY,color_BL,color_BK,color_WT,color_BL,color_SL,route_00600,route_00500,cluster
Make,,,,,,,,,,,,,
ACUR,0.001659,-0.003236,-0.001964,0.001590,-0.001858,0.002586,-0.009768,0.009383,0.002586,-0.006129,-0.008421,-0.001067,0
TOYO,-0.020102,-0.002450,0.019959,-0.003473,-0.001646,0.002974,0.008510,0.000856,0.002974,-0.000633,-0.006795,-0.002440,0
NISS,-0.001246,-0.000193,-0.000494,-0.003980,-0.006371,0.003191,-0.000990,-0.000195,0.003191,0.009521,0.003793,0.000070,0
INFI,0.012707,-0.003742,0.009365,0.006225,-0.001384,0.007859,0.002304,0.003941,0.007859,-0.006404,-0.006553,0.008646,0
HYUN,-0.000987,-0.000440,0.001563,0.006151,-0.000142,0.005452,0.001276,-0.010277,0.005452,-0.002758,-0.000288,0.000713,0
HOND,0.001198,-0.000484,0.001304,-0.003177,-0.002195,0.002671,0.001417,0.000947,0.002671,-0.001682,0.001698,-0.000076,0
GMC,-0.002490,-0.001147,-0.001535,-0.006107,0.003075,0.006808,-0.000285,-0.012287,0.006808,0.006798,0.001806,0.016590,0
JEEP,0.004187,-0.001816,0.001122,0.003216,0.000420,0.008180,-0.002034,-0.003658,0.008180,0.003661,-0.000489,0.002277,0
DODG,-0.006523,-0.002494,-0.001492,0.000095,-0.001184,0.007648,-0.003339,-0.002573,0.007648,0.000241,-0.011637,0.003638,0


In [24]:
df_group.groupby('cluster').mean()
        

,Fine amount,plate_expiration_diff_ts,lat_long_outlier_score,route_dummy_replacement_value,color_GY,color_BL,color_BK,color_WT,color_BL,color_SL,route_00600,route_00500
cluster,,,,,,,,,,,,
0,-0.002535,-0.001319,0.001831,0.001300,-0.000460,0.006628,-0.001602,-0.003095,0.006628,-0.000237,-0.002811,0.002261
1,-0.000691,-0.000957,-0.000190,-0.000452,0.002798,-0.004462,-0.000812,0.000889,-0.004462,-0.000311,0.000867,-0.000840


# Features

There are 3 general type of features: location, car characteristics and ticket/violation characteristics. 

The best location features are the Route and ticket location density (lat_long_outlier_score). 

The best car characteristics features are the Body type and the car color. 

The best violation features are the fine amount and datetime information about the violation. Fine amount is important as it captures other violation information and is correlated to violation description features.

In [88]:
df_copy = df.copy()
df_copy = df_copy.dropna(subset = ['Make'])
df_copy['target'] = df_copy['Make'].isin(top_25_makes).astype(int)

def pad_int(num, l = 4):
    if not pd.isna(num):
        num = int(float(num))
        num_str = str(num)
        while len(num_str) < l:
            num_str  = '0' + num_str
        return num_str

df_copy['ticket_dt'] = pd.to_datetime(df_copy.loc[:,'Issue Date'], errors='coerce')
df_copy['ticket_year'] = df_copy['ticket_dt'].dt.year
df_copy['ticket_month'] = df_copy['ticket_dt'].dt.month
df_copy['ticket_dow'] = df_copy['ticket_dt'].dt.dayofweek
df_copy['ticket_hour_of_day'] = df_copy['Issue time'].apply(lambda x: pad_int(x)).astype(str).str[:2]


In [89]:
def get_cat_info_about_column(df, col_name, min_perc = .01):
    valid_values = [i for  i, j in dict(df_copy[col_name].value_counts(normalize = True)).items() if j >= min_perc]
    df_valid = df_copy[df_copy[col_name].isin(valid_values)]
    mean_df =  df_valid.groupby([col_name])['target'].mean().to_frame()
    count_df = df_valid[col_name].value_counts(normalize = True).to_frame()
    count_df =  count_df.reset_index()
    count_df.columns = [col_name, 'perc_of_values']
    mean_df.columns = ['mean_target']
    mean_df =  mean_df.reset_index()
    output = mean_df.merge(count_df)
    return output.sort_values('perc_of_values', ascending = False)
    

In [90]:
get_cat_info_about_column(df_copy, 'RP State Plate', min_perc = .001)


,RP State Plate,mean_target,perc_of_values
1,CA,0.914914,0.943410
0,AZ,0.939867,0.009045
19,TX,0.923509,0.006372
12,NV,0.938159,0.006059
3,FL,0.908506,0.004441
22,WA,0.936398,0.004020
5,IL,0.904017,0.002899
16,OR,0.926637,0.002807
2,CO,0.939125,0.002566
13,NY,0.909765,0.001911


In [91]:
get_cat_info_about_column(df_copy, 'Route', min_perc = .01)


,Route,mean_target,perc_of_values
8,00600,0.906531,0.292103
7,00500,0.915932,0.205643
5,00402,0.931876,0.099345
4,00401,0.924408,0.096670
6,00403,0.935765,0.074387
3,00315,0.917117,0.063738
1,00111,0.904189,0.061456
0,00107,0.914930,0.056316
2,00309,0.922919,0.050343


In [92]:
get_cat_info_about_column(df_copy, 'Body Style', min_perc = .01)


,Body Style,mean_target,perc_of_values
1,PA,0.927128,0.896611
2,PU,0.992731,0.034937
4,VN,0.925558,0.027223
3,TK,0.686678,0.024821
0,CM,0.651436,0.016408


In [93]:
get_cat_info_about_column(df_copy, 'ticket_hour_of_day', min_perc = .01)

,ticket_hour_of_day,mean_target,perc_of_values
10,12,0.916145,0.129633
6,08,0.928770,0.126963
8,10,0.917084,0.123134
9,11,0.904551,0.087580
7,09,0.906974,0.063702
11,13,0.904565,0.063257
14,16,0.904276,0.053326
12,14,0.900727,0.048345
16,18,0.922039,0.038526
15,17,0.911528,0.037841


In [99]:
# get_cat_info_about_column(df_copy, 'ticket_month', min_perc = .01)


In [98]:
get_cat_info_about_column(df_copy, 'ticket_dow', min_perc = .01)


,ticket_dow,mean_target,perc_of_values
1,1,0.912119,0.194048
3,3,0.915282,0.187689
2,2,0.915017,0.185382
0,0,0.912202,0.163238
4,4,0.914962,0.162515
5,5,0.920945,0.056258
6,6,0.923178,0.050870


In [100]:
df_corr = df_labels.corr()



In [105]:

df_corr[(df_corr['Fine amount'] > .1)|(df_corr['Fine amount'] < -.1)].sort_values('Fine amount')

,Unnamed: 0,rp_state_plate_CA,rp_state_plate_dummy_replacement_value,color_BK,color_BL,color_BN,color_GN,color_GO,color_GY,color_MR,...,ticket_hour_of_day_84,ticket_hour_of_day_85,ticket_hour_of_day_90,ticket_hour_of_day_91,ticket_hour_of_day_92,ticket_hour_of_day_93,ticket_hour_of_day_dummy_replacement_value,Fine amount,lat_long_outlier_score,plate_expiration_diff_ts
violation_code_5204A-,0.000850,0.067625,-0.067625,0.001434,0.005625,-0.020215,0.019858,0.010108,-0.005932,0.009860,...,-0.007009,-0.003329,0.000067,-0.000411,0.000168,0.000698,0.032757,-0.359311,-0.007185,0.033760
violation_desc_DISPLAY OF TABS,0.000850,0.067625,-0.067625,0.001434,0.005625,-0.020215,0.019858,0.010108,-0.005932,0.009860,...,-0.007009,-0.003329,0.000067,-0.000411,0.000168,0.000698,0.032757,-0.359311,-0.007185,0.033760
violation_code_5200,0.001174,0.043663,-0.043663,0.032761,-0.003209,-0.015704,-0.009995,-0.006869,-0.000933,-0.006351,...,-0.007870,-0.004715,-0.006610,-0.006346,-0.003394,-0.002470,0.004578,-0.235360,0.007658,0.013699
violation_desc_DISPLAY OF PLATES,0.001174,0.043663,-0.043663,0.032761,-0.003209,-0.015704,-0.009995,-0.006869,-0.000933,-0.006351,...,-0.007870,-0.004715,-0.006610,-0.006346,-0.003394,-0.002470,0.004578,-0.235360,0.007658,0.013699
violation_code_88.13B+,0.000272,-0.017319,0.017319,0.037732,-0.006868,-0.026866,-0.021376,-0.015220,0.007799,-0.013957,...,-0.037499,-0.029477,-0.024442,-0.023434,-0.015649,-0.007252,-0.115937,-0.102341,0.064992,-0.001926
violation_desc_METER EXP.,0.000272,-0.017319,0.017319,0.037732,-0.006868,-0.026866,-0.021376,-0.015220,0.007799,-0.013957,...,-0.037499,-0.029477,-0.024442,-0.023434,-0.015649,-0.007252,-0.115937,-0.102341,0.064992,-0.001926
violation_code_dummy_replacement_value,0.001037,-0.000592,0.000592,-0.019691,0.000640,-0.004275,-0.017782,0.003598,-0.018815,-0.009700,...,-0.023646,-0.014163,-0.008383,-0.008449,-0.004537,0.002631,0.080723,0.170950,-0.096937,0.016831
violation_code_80.56E4+,-0.000532,0.007586,-0.007586,-0.029748,-0.017079,0.192555,-0.001236,-0.003901,-0.027349,-0.002045,...,-0.021811,-0.015334,-0.011074,-0.009652,-0.001989,0.003892,0.089296,0.194697,0.004313,-0.081224
violation_desc_RED ZONE,-0.000319,0.007672,-0.007672,-0.029292,-0.016353,0.184740,-0.004084,-0.003688,-0.026498,-0.003813,...,-0.022215,-0.015594,-0.011510,-0.010277,-0.002509,0.003527,0.098384,0.201302,0.004329,-0.075618
violation_desc_dummy_replacement_value,0.001081,-0.001228,0.001228,-0.019449,-0.000366,-0.003133,-0.013458,0.002873,-0.017922,-0.006376,...,-0.025268,-0.016150,-0.011377,-0.010729,-0.007311,-0.000462,0.076871,0.253882,-0.093626,0.012552


In [106]:
df_corr[(df_corr['plate_expiration_diff_ts'] > .1)|(df_corr['plate_expiration_diff_ts'] < -.1)].sort_values('plate_expiration_diff_ts')

,Unnamed: 0,rp_state_plate_CA,rp_state_plate_dummy_replacement_value,color_BK,color_BL,color_BN,color_GN,color_GO,color_GY,color_MR,...,ticket_hour_of_day_84,ticket_hour_of_day_85,ticket_hour_of_day_90,ticket_hour_of_day_91,ticket_hour_of_day_92,ticket_hour_of_day_93,ticket_hour_of_day_dummy_replacement_value,Fine amount,lat_long_outlier_score,plate_expiration_diff_ts
color_BN,0.000476,0.022457,-0.022457,-0.074064,-0.041042,1.000000,-0.024349,-0.016344,-0.069358,-0.015561,...,-0.012134,-0.009220,-0.008138,-0.005241,0.002886,0.010739,-0.014765,0.045617,0.019247,-0.162739
body_style_CM,0.001341,0.011131,-0.011131,-0.055906,-0.023681,0.277294,-0.005991,-0.011964,-0.054591,-0.009842,...,-0.011483,-0.009512,-0.007439,-0.007421,-0.000050,0.006785,-0.012232,0.047884,0.008748,-0.123776
body_style_PA,0.000359,-0.010378,0.010378,0.094720,0.023830,-0.215208,-0.005252,0.011019,0.083599,0.003878,...,0.006101,0.002485,-0.003065,-0.003939,-0.012139,-0.015897,-0.032826,-0.051081,0.050282,0.111827
plate_expiration_diff_ts,-0.000687,0.076915,-0.076915,0.003964,0.017584,-0.162739,0.012109,0.014463,0.025608,0.008962,...,-0.001075,-0.003311,-0.002805,-0.005036,-0.010027,-0.013517,0.005818,-0.031614,0.002953,1.000000


# Evaluation

In [ ]:
24072 269106 13273 129304